In [1]:
import os
import keras
import pickle
import numpy as np
import pandas as pd
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
import tensorflow as tf

Using TensorFlow backend.


In [0]:
files = os.listdir('/content/drive/My Drive/dataset_masters/')
files = ['/content/drive/My Drive/dataset_masters/'+f for f in files if "csv" in f]

In [0]:
def build_model(input_shape):
  model = keras.models.Sequential([keras.layers.Dense(128,activation='relu',input_shape=input_shape),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(128,activation='relu'),
                                   keras.layers.Dense(1,activation='sigmoid')])
  return model

def run_nn(x_train, y_train, x_test, y_test, batch_size, epochs):
  model = build_model((4,))
  model.compile(optimizer='adam', loss="binary_crossentropy", metrics=['binary_accuracy'])
  model.fit(x_train, y_train, batch_size=batch_size, epochs=epochs, verbose=0)
  scores = model.evaluate(x_test,y_test)
  print("NN ", scores[1])
  return model, scores[1]

In [5]:
min_max = {}
for f in files:
  data = pd.read_csv(f)
  data['STATUS1'] = data['STATUS'].apply(lambda x: 1 if x==2 else 0)
  x, y = data.iloc[:,:4], data.iloc[:,-1]
  
  # Normalization
  mins = dict(x.min())
  maxs = dict(x.max())
  univ = f.split('/')[-1].split('.')[0].split('_')[1]
  min_max[univ] = {}
  min_max[univ]['min'] = mins
  min_max[univ]['max'] = maxs
  print(univ)
  
  x = (x-x.min())/(x.max()-x.min())
  
  fold = 15
  best_score, best_model = {'KNN': -1, 'NN': -1}, {'KNN': None, 'NN': None}
  for i in range(fold):
    print("Fold ", i)
    x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7)

    # Scikit Models
    clf = KNeighborsClassifier(2)
    clf.fit(x_train, y_train)
    knn_score = clf.score(x_test, y_test)
    print("KNN ", knn_score)

    model, nn_score = run_nn(x_train, y_train, x_test, y_test, 32, 250)

    if knn_score > best_score['KNN']:
      best_score['KNN'] = knn_score
      best_model['KNN'] = clf
    
    if nn_score > best_score['NN']:
      best_score['NN'] = nn_score
      best_model['NN'] = model

  best_model['NN'].save('/content/drive/My Drive/dataset_masters/models/'+univ+'.h5')
  pickle.dump(best_model['KNN'], open('/content/drive/My Drive/dataset_masters/models/'+univ+'.pkl','wb'))
pickle.dump(min_max, open('/content/drive/My Drive/dataset_masters/min_max.pkl','wb'))

usc
Fold  0
KNN  0.642967542503864
647/647 [==============================] - 0s 95us/step
NN  0.874806821346283
Fold  1
KNN  0.6306027820710973
647/647 [==============================] - 0s 91us/step
NN  0.8485316634178162
Fold  2
KNN  0.6738794435857806
647/647 [==============================] - 0s 123us/step
NN  0.8299845457077026
Fold  3
KNN  0.6491499227202473
647/647 [==============================] - 0s 107us/step
NN  0.8516228795051575
Fold  4
KNN  0.6553323029366306
647/647 [==============================] - 0s 96us/step
NN  0.8562596440315247
Fold  5
KNN  0.6506955177743431
647/647 [==============================] - 0s 101us/step
NN  0.755795955657959
Fold  6
KNN  0.6491499227202473
647/647 [==============================] - 0s 96us/step
NN  0.8887171745300293
Fold  7
KNN  0.6197836166924265
647/647 [==============================] - 0s 92us/step
NN  0.8485316634178162
Fold  8
KNN  0.6321483771251932
647/647 [==============================] - 0s 124us/step
NN  0.8778979778289

In [10]:
from sklearn.svm import SVC
from sklearn.tree import ExtraTreeClassifier
from sklearn.ensemble import RandomForestClassifier

for f in files:
  data = pd.read_csv(f)
  data['STATUS1'] = data['STATUS'].apply(lambda x: 1 if x==2 else 0)
  x, y = data.iloc[:,:4], data.iloc[:,-1]
  
  # Normalization
  univ = f.split('/')[-1].split('.')[0].split('_')[1]
  print(univ)
  
  x = (x-x.min())/(x.max()-x.min())
  
  fold = 15
  names = ['KNN', 'ET', 'RF', 'SVC']
  estimators = [KNeighborsClassifier(2), ExtraTreeClassifier(), RandomForestClassifier(), SVC()]
  best_score, best_model = {k: -1 for k in names}, {k: None for k in names}
  for i in range(fold):
    print("Fold ", i)
    x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7)

    # Scikit Models
    for name, clf in zip(names,estimators):
      clf.fit(x_train, y_train)
      score = clf.score(x_test, y_test)
      print(name, score)
      if score > best_score[name]:
        best_score[name] = score
        best_model[name] = clf

  for name in names:
    pickle.dump(best_model[name], open('/content/drive/My Drive/dataset_masters/models/'+name+'_'+univ+'.pkl','wb'))
# pickle.dump(min_max, open('/content/drive/My Drive/dataset_masters/min_max.pkl','wb'))

usc
Fold  0
KNN 0.6306027820710973
ET 0.7882534775888718
RF 0.8902627511591963
SVC 0.6692426584234931
Fold  1
KNN 0.6738794435857806
ET 0.8083462132921174
RF 0.8732612055641422
SVC 0.6275115919629057
Fold  2
KNN 0.6862442040185471
ET 0.8469860896445132
RF 0.8964451313755796
SVC 0.6615146831530139
Fold  3
KNN 0.6553323029366306
ET 0.7897990726429676
RF 0.8639876352395672
SVC 0.6228748068006182
Fold  4
KNN 0.6939721792890263
ET 0.7928902627511591
RF 0.9010819165378671
SVC 0.6769706336939721
Fold  5
KNN 0.6769706336939721
ET 0.8346213292117465
RF 0.8979907264296755
SVC 0.6646058732612056
Fold  6
KNN 0.6568778979907264
ET 0.7573415765069552
RF 0.8763523956723338
SVC 0.6846986089644513
Fold  7
KNN 0.6476043276661515
ET 0.7882534775888718
RF 0.8825347758887172
SVC 0.6584234930448223
Fold  8
KNN 0.6506955177743431
ET 0.794435857805255
RF 0.8809891808346213
SVC 0.6553323029366306
Fold  9
KNN 0.6506955177743431
ET 0.8191653786707882
RF 0.865533230293663
SVC 0.6584234930448223
Fold  10
KNN 0.624

In [22]:
from sklearn.linear_model import LogisticRegression

for f in files:
  data = pd.read_csv(f)
  data['STATUS1'] = data['STATUS'].apply(lambda x: 1 if x==2 else 0)
  x, y = data.iloc[:,:4], data.iloc[:,-1]
  
  # Normalization
  univ = f.split('/')[-1].split('.')[0].split('_')[1]
  print(univ)
  
  x = (x-x.min())/(x.max()-x.min())
  
  names = ['KNN', 'ET', 'RF', 'SVC']
  estimators = [pickle.load(open('/content/drive/My Drive/dataset_masters/models/'+name+'/'+name+'_'+univ+'.pkl','rb')) for name in names]
  # estimators.append(keras.models.load_model('/content/drive/My Drive/dataset_masters/models/NN/'+univ+'.h5'))
  # names.append('NN')
  x_train, x_test, y_train, y_test = train_test_split(x,y,train_size=0.7)

  preds = []
  for name, clf in zip(names,estimators):
    pred = clf.predict(x_train).reshape((-1,1))
    preds.append(pred)

  est = LogisticRegression()
  est.fit(np.hstack(tuple(preds)),y_train)
  
  preds = []
  for name, clf in zip(names,estimators):
    pred = clf.predict(x_test).reshape((-1,1))
    preds.append(pred)
  
  score = est.score(np.hstack(tuple(preds)),y_test)
  print(score)
  if 'NN' in names:
    name = 'LR_with_NN'
  else:
    name = 'LR_without_NN'
  pickle.dump(est,open('/content/drive/My Drive/dataset_masters/models/'+name+'/'+name+'_'+univ+'.pkl','wb'))

usc
0.9690880989180835
ncsu
0.9045936395759717
neu
0.9101449275362319
mit
1.0
utd
0.9424131627056673
